In [ ]:
# install required packages
pip install tensorflow
pip install keras_tuner


In [ ]:
# import required packages
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Flatten, Convolution2D, BatchNormalization
from tensorflow.keras.layers import ReLU, MaxPool2D, AvgPool2D, GlobalAvgPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
import keras_tuner as kt
from sklearn.model_selection import train_test_split


In [ ]:
# load the CIFAR-10 dataset from keras
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()


# Normalize the image pixel values
img_train = x_train.astype('float32') / 255.0
img_test = x_test.astype('float32') / 255.0

# split the train data into train and validation sets
x_train, y_train, x_val, y_val = train_test_split(
    x_train, y_train,                                                             test_size=0.25)


In [ ]:
# function to build an hypermodel
# takes an argument from which to sample hyperparameters
def build_model(hp):
​
  inputs = Input(shape = (32, 32, 3)) #input layer
  x = inputs
​
  # iterate a number of conv blocks from min_value to max_value
  # tune the number of filters
  # choose an optimal value from min_value to max_value
  for i in range(hp.Int('conv_blocks',min_value = 3, max_value = 5, default=3)): # Int specifies the dtype of the values
    filters = hp.Int('filters_' + str(i),min_value = 32,max_value = 256, step=32) 
​
    for _ in range(2):
      # define the conv, BatchNorm and activation layers for each block
      x = Convolution2D(filters, kernel_size=(3, 3), padding= 'same')(x)
      x = BatchNormalization()(x)
      x = ReLU()(x)
​
    # choose an optimal pooling type
    if hp.Choice('pooling_' + str(i), ['avg', 'max']) == 'max': # hp.Choice chooses from a list of values
        x = MaxPool2D()(x)
    else:
        x = AvgPool2D()(x)
​
  x = GlobalAvgPool2D()(x) # apply GlobalAvG Pooling
​
  # Tune the number of units in the  Dense layer
  # Choose an optimal value between min_value to max_value
  x = Dense(hp.Int('Dense units',min_value = 30, max_value = 100, step=10, default=50), activation='relu')(x)
  outputs = Dense(10, activation= 'softmax')(x) # output layer 
  
  # define the model
  model = Model(inputs, outputs)
​
  # Tune the learning rate for the optimizer
  # Choose an optimal value frommin_value to max_value
  model.compile(optimizer= Adam(hp.Float('learning_rate', min_value = 1e-4, max_value =1e-2, sampling='log')), 
                loss= 'sparse_categorical_crossentropy', metrics = ['accuracy'])
  return model

In [ ]:
# initialize tuner to run the model.
# using the Hyperband search algorithm
tuner = kt.Hyperband(
    hypermodel=build_model,
    objective='val_accuracy',
    max_epochs=30,
    hyperband_iterations=2,
    directory="Keras_tuner_dir",
    project_name="Keras_tuner_Demo")


In [ ]:
# Initialize a random search tuner
# using the Resnet architecture
# and the Random Search algorithm
tuner = kt.tuners.RandomSearch(
    kt.applications.HyperResNet(input_shape=(32, 32, 3), classes=10),
    objective='val_accuracy',
    max_trials=30)


In [ ]:
# Run the search
tuner.search(x_train, y_train,
             validation_data=(x_test, y_test),
             epochs=30,
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=2)])


In [ ]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(1)[0]

# get the best model
best_model = tuner.get_best_models(1)[0]


In [ ]:
nblocks = best_hps.get('conv_blocks')
print(f'Number of conv blocks: {nblocks}')
for hyparam in [f'filters_{i}' for i in range(nblocks)] + [f'pooling_{i}' for i in range(nblocks)] + ['Dense units'] + ['learning_rate']:
    print(f'{hyparam}: {best_hps.get(hyparam)}')


In [ ]:
# display model structure
plot_model(best_model, 'best_model.png', show_shapes=True)

# show model summary
best_model.summary()


# Training the model

In [ ]:
# Build the model with the optimal hyperparameters
# train the model.
model = tuner.hypermodel.build(best_hps)
model.fit(x_train, y_train, 
          validation_data= (x_val,y_val), 
          epochs= 25,
           callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)])

# Evaluate the result

In [ ]:
# evaluate the result
eval_result = model.evaluate(x_test, y_test)
print(f"test loss: {eval_result[0]}, test accuracy: {eval_result[1]}")